Homework 11: Clustering - https://github.com/fedhere/PUI2016_fb55/tree/master/HW11_fb55
Author: fhl204

In [10]:
from __future__ import print_function
import numpy as np
import pandas as pd
import pylab as pl
import os
%pylab inline
import geopandas as gpd
from geopandas import GeoDataFrame

from fiona.crs import from_epsg
import shapely
from shapely.geometry import Point

import seaborn
from descartes import PolygonPatch

os.getenv("PUIDATA")

Populating the interactive namespace from numpy and matplotlib


/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


'/home/cusp/fhl204/PUIdata'

In [11]:
#load Census Tract Shape File: nycb2010_16d
def download_nycb2010_16d_zip():
    os.system("curl -O http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nycb2010_16d.zip") 
    os.system("mv " + "nycb2010_16d.zip " + os.getenv("PUIDATA")+"/nycb2010_16d.zip")
    os.system("unzip " + os.getenv("PUIDATA")  + "/nycb2010_16d.zip -d " + os.getenv("PUIDATA"))
    os.system("rm " + os.getenv("PUIDATA") + "/nycb2010_16d.zip")

In [15]:
shape_path = os.getenv("PUIDATA") + '/nycb2010.shp'
ct_shape = GeoDataFrame.from_file(shape_path)

IOError: Unable to open /home/cusp/fhl204/PUIdata/nycb2010.shx or /home/cusp/fhl204/PUIdata/nycb2010.SHX.Try --config SHAPE_RESTORE_SHX true to restore or create it

In [7]:
ct_shape.head()

NameError: name 'ct_shape' is not defined

In [8]:
ct_latlon = ct_shape[ct_shape.BoroName == 'Brooklyn']

NameError: name 'ct_shape' is not defined

In [ ]:
ct_latlon = ct_latlon.to_crs(epsg=4326)
cusp_coor = [-73.9866, 40.6931]

In [ ]:
cusp_shapely_geometry = Point(cusp_coor)

In [ ]:
for i in ct_latlon.index:
    if shapely.geometry.asShape(ct_latlon.geometry[i]).contains(cusp_shapely_geometry):
        index = i
        ct = ct_latlon.geometry[i]
        print('CUSP is contained in', ct)

In [ ]:
ct_latlon_10 = ct_latlon.iloc[::10]

In [ ]:
ct_shape.crs = from_epsg(4326) # epsg=4326: lat/on | 26918: NAD83/UTM zone 18N | epsg=2263 is US feet
NYC_Area = ct_shape.to_crs(epsg=2263).geometry.area.sum()/(2.788*10**7) # from ft2 to mi2 
print 'total NYC land area: %s (mi^2)'%str(NYC_Area)[:5] # from ft2 to mi2 #epsg=2263 is US feet

# a view of the shape file loaded as a geojson file will look like this. Notice the coordinates. They are in 
# epsg=2263 is US feet. You want to convert them in epsg=4326: lat/on . 
# Look at the notebook https://github.com/fedhere/PUI2016_fb55/blob/master/Lab9_SRK325/GeospatialAnalysis_CitiBike.ipynb
ct_shape.head()

In [ ]:
fig, ax = pl.subplots(figsize=(10,8))
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ct_latlon_10.geometry.plot(cmap='Greys', alpha=1, linewidth=0.5, ax=ax)
ax.scatter(cusp_shapely_geometry.x, cusp_shapely_geometry.y, c='blue', alpha=0.5, s=200)
ct_patch = PolygonPatch(ct, fc='red', ec='red', alpha=1)
ax.add_patch(ct_patch)
ax.set_xlim(-74.05, -73.83)
ax.set_ylim(40.56, 40.74)

### Figure 1: Brooklyn Polygons with CUSP in Blue at the Census Tract Polygon in Red